In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load data
df = pd.read_csv("greaternoida_df.csv")
df = df.dropna(subset=["Price (in lakhs)"])

# Log-transform the target to reduce skew
y = np.log1p(df["Price (in lakhs)"])
X = df.drop(columns=["Price (in lakhs)", "Index"])

# Feature groups
numerical_cols = ["Floor (Ratio)", "Bathroom", "Balcony", "BHK", "Final Carpet Area (in sqft)"]
categorical_cols = ["Furnishing", "facing", "Ownership", "Car Parking", "Transaction", "Garden/Park", "Main Road", "Pool"]

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_cols),
    ('cat', cat_pipeline, categorical_cols)
])

# Initial model pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Extract important features
model = pipeline.named_steps['model']
ohe = pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
ohe_feature_names = ohe.get_feature_names_out(categorical_cols)
feature_names = numerical_cols + list(ohe_feature_names)
importances = model.feature_importances_

# Filter important features (threshold: > 0.01)
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
important_features = importance_df[importance_df['importance'] > 0.00]['feature'].tolist()

# Map back to original column names
important_numerical = [col for col in numerical_cols if col in important_features]
important_categorical = [col for col in categorical_cols if any(col in f for f in important_features)]

print("Selected Numerical Columns:", important_numerical)
print("Selected Categorical Columns:", important_categorical)

# Redefine preprocessors with reduced columns
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, important_numerical),
    ('cat', cat_pipeline, important_categorical)
])

# Final model pipeline
final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train on reduced features
X_train = X_train[important_numerical + important_categorical]
X_test = X_test[important_numerical + important_categorical]
final_pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred_log = final_pipeline.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_actual = np.expm1(y_test)

rmse = np.sqrt(mean_squared_error(y_actual, y_pred))
r2 = r2_score(y_actual, y_pred)
relative_rmse = rmse / y_actual.mean()

print(f"RMSE: {rmse:.2f}")
print(f"Relative RMSE: {relative_rmse:.2f}")
print(f"R² Score: {r2:.2f}")


Selected Numerical Columns: ['Floor (Ratio)', 'Bathroom', 'Balcony', 'BHK', 'Final Carpet Area (in sqft)']
Selected Categorical Columns: ['Furnishing', 'facing', 'Ownership', 'Car Parking', 'Transaction', 'Garden/Park', 'Main Road', 'Pool']
RMSE: 45.03
Relative RMSE: 0.61
R² Score: 0.44


In [3]:
import os
import joblib

# Create folder if it doesn't exist
os.makedirs("website/models", exist_ok=True)

# Save the full pipeline — not just the model
joblib.dump(final_pipeline, "website/models/greaternoida_model.pkl")

['website/models/greaternoida_model.pkl']

In [4]:
# --- INFERENCE ---
import joblib
import pandas as pd

# Load the full pipeline
pipeline = joblib.load("website/models/greaternoida_model.pkl")

# Extract preprocessor to get used columns
preprocessor = pipeline.named_steps["preprocessor"]
used_num_cols = preprocessor.transformers_[0][2]
used_cat_cols = preprocessor.transformers_[1][2]
used_columns = used_num_cols + used_cat_cols

# Full input from user (can include unused features — that's okay)
input_data = {
    "Floor (Ratio)": 0.5,
    "Bathroom": 2,
    "Balcony": 1,
    "Car Parking": 1,
    "BHK": 2,
    "Final Carpet Area (in sqft)": 950,
    "Furnishing": "Furnished",
    "facing": "East",
    "Ownership": "Freehold",
    "Transaction": "Resale",
    "Garden/Park": 1,
    "Main Road": 1,
    "Pool": 0,
    "Not Available": 0  # Extra column, will be ignored
}

input_df = pd.DataFrame([input_data])
input_df_filtered = input_df[used_columns]

# Predict
log_pred = pipeline.predict(input_df_filtered)[0]
predicted_price = np.expm1(log_pred)  # Correct way to invert log1p
print(f"Predicted Price: ₹{predicted_price:.2f} lakhs")

Predicted Price: ₹55.15 lakhs


In [5]:
df


,Index,Price (in lakhs),City,Floor (Ratio),Furnishing,facing,Bathroom,Balcony,Car Parking,Ownership,BHK,Location,Final Carpet Area (in sqft),CurrentFloor,Transaction,Garden/Park,Main Road,Not Available,Pool
0,0,55.0,greater-noida,0.619048,2,-1,2,3,-1,-1,2.0,Ace Platinum Zeta 1,920.0,13.0,1,0,1,0,0
1,1,80.0,greater-noida,0.000000,1,-1,3,0,0,-1,3.0,Sushant Megapolis Greater Noida,1338.4,0.0,0,0,0,0,0
2,2,79.6,greater-noida,0.500000,1,45,2,4,-1,3,3.0,AIG Royal Greater Noida West,1057.0,10.0,0,1,1,0,1
3,3,69.8,greater-noida,0.500000,1,45,2,3,-1,3,2.0,AIG Royal Greater Noida West,968.0,10.0,0,1,1,0,1
4,4,70.0,greater-noida,0.473684,1,90,2,2,1,3,2.0,CGEWHO Phase 1 Phi 2,1120.0,9.0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4705,4705,82.0,greater-noida,0.739130,1,-1,3,3,-1,-1,3.0,Edana Alpha 1,1050.0,17.0,0,0,0,0,0
4706,4706,149.0,greater-noida,0.444444,1,-1,3,0,-1,-1,3.0,Jaypee Greens,1430.4,4.0,0,0,0,0,0
4707,4707,46.0,greater-noida,0.560000,1,180,2,3,1,3,2.0,Greater Noida West,1050.0,14.0,0,1,1,0,0
4708,4708,112.0,greater-noida,0.777778,1,45,3,3,1,3,3.0,Sector 10,980.0,21.0,0,1,0,0,0
